## Wprowadzenie do **dbzero** (2/12). Listy, relacje i singletony.

In [1]:
import dbzero as db0
db0.init(dbzero_root = "/dbzero", prefix="data")

Musimy ponownie zdefiniować naszą klasę. W typowym programie Python po prostu zaimportowalibyśmy ją z modułu.

In [2]:
@db0.memo
class HelloWorld:
    def __init__(self, name):
        self.name = name
    
    def greet(self):
        print(f"Witaj {self.name}. Witamy w dbzero!!")

I otwórzmy jej instancję za pomocą wcześniej wygenerowanego UUID (< wklej swoje UUID poniżej >)

In [3]:
h = db0.fetch(HelloWorld, "DFZ46RQG7QPR3IEBUCAIAAIN")

RuntimeError: Prefix: 2098673055265944345 not found


In [4]:
h.greet()

NameError: name 'h' is not defined

### W porządku, to zadziałało. Instancja pamięta swoje elementy danych.
Ale pobieranie po ID nie wydaje się zbyt wygodne, jak mogę uzyskać dostęp do moich obiektów w bardziej praktyczny sposób?

Jedną z możliwości jest przechowywanie odniesień do obiektów w innym (nadrzędnym) obiekcie. W świecie baz danych takie powiązanie nazywane byłoby 'relacją'. Jeśli naszym zamiarem jest przechowywanie tylko jednej instancji HelloMany, możemy oznaczyć ją jako

In [5]:
@db0.memo(singleton = True)
class HelloMany:
    # przekazujemy listę imion i zostanie utworzonych wiele instancji HelloWorld, które następnie zostaną dodane do listy DB0 'hellos'
    def __init__(self, names):
        self.hellos = [HelloWorld(name) for name in names]
        
    def greet(self):
        for hw in self.hellos:
            hw.greet()

Zwróć uwagę, że obiekt **dbzero** klasy oznaczonej jako ``singleton`` zostanie utworzony tylko raz.

In [6]:
many_hellos = HelloMany(["Andrzej", "Alicja", "Bartek", "Filip", "Robert"])

In [7]:
many_hellos.greet()

Witaj Andrzej. Witamy w dbzero!!
Witaj Alicja. Witamy w dbzero!!
Witaj Bartek. Witamy w dbzero!!
Witaj Filip. Witamy w dbzero!!
Witaj Robert. Witamy w dbzero!!


Po utworzeniu instancji obiekt można ponownie uzyskać dostęp (z innego programu lub po zamknięciu tego programu) bez żadnych argumentów.

In [8]:
mh = HelloMany()
mh.greet()

Witaj Andrzej. Witamy w dbzero!!
Witaj Alicja. Witamy w dbzero!!
Witaj Bartek. Witamy w dbzero!!
Witaj Filip. Witamy w dbzero!!
Witaj Robert. Witamy w dbzero!!


Widzimy, że many_hellos i mh to ten sam obiekt ``dbzero`` i nawet ten sam obiekt Python.

In [9]:
print(f"{id(many_hellos)} {id(mh)}")

140401140747472 140401140747472


In [10]:
print(f"{db0.uuid(many_hellos)} {db0.uuid(mh)}")

UCEZKEPIYX26NIEBUHJIABYN UCEZKEPIYX26NIEBUHJIABYN


### Co więc jest faktycznie przechowywane na liście HelloMany.hellos?
Podobnie jak w przypadku zwykłych obiektów Python - lista przechowuje odniesienia, a nie pełne instancje. Listy **dbzero** mogą przechowywać proste typy (liczby/ciągi znaków), inne kolekcje (listy, słowniki, zbiory, krotki itp.) lub odniesienia do innych instancji dbzero ``@memo``.

In [ ]:
for obj in mh.hellos:
    print(db0.uuid(obj))

A lista **dbzero** działa dokładnie jak każda zwykła lista Python - możesz iterować, wycinać lub uzyskiwać dostęp do jej elementów przez indeks.

In [ ]:
mh.hellos[3].greet()

### Jak duża może być pojedyncza lista?
W Python normalnie twoja lista byłaby ograniczona przez rozmiar pamięci RAM dostępnej dla twojego procesu (w praktyce, podczas uruchamiania programu - kto wie, jaki jest dokładnie ten limit???). Dla użytkowników DB0 (chmurowych) mamy dobrą wiadomość. Nie ma limitów! Jeśli chcesz, możesz utworzyć listę o rozmiarze znacznie przekraczającym dostępną pamięć. W przypadku wersji lokalnej ten rozmiar jest ograniczony przez dostępną przestrzeń dyskową.

In [ ]:
print(len(mh.hellos))

In [ ]:
for i in range(1000):
    mh.hellos.append(HelloWorld(f"imię-{i}"))

In [ ]:
print(len(mh.hellos))

Zbadajmy wykorzystanie pamięci i limity w następnym notebooku.

In [ ]:
db0.close()